In [ ]:
!pip install qiskit qutip numpy matplotlib torch tqdm scikit-learn einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from qutip import basis, ket2dm, Qobj

zero = basis(2, 0)  # |0> = [1, 0]
one = basis(2, 1)   # |1> = [0, 1]

# Example: create a pure superposition state
psi = (zero + one).unit()  # |ψ> = (|0> + |1>)/√2
pure_dm = ket2dm(psi)      # Density matrix ρ = |ψ><ψ|

# Create a mixed state as a weighted sum of pure states
p = 0.7213214678631  # probability of being |ψ>
rho_mixed = p * pure_dm + (1 - p) * ket2dm(one)

print("Mixed-state density matrix:")
print(rho_mixed)

Mixed-state density matrix:
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.36066073 0.36066073]
 [0.36066073 0.63933927]]


In [ ]:
from qutip import basis, ket2dm, Qobj
import numpy as np

def mixed_state_generation(n_samples=10000):
    mixed_states = []
    for _ in range(n_samples):
        # Random angles on Bloch sphere
        theta = np.arccos(2*np.random.rand() - 1)  # polar angle [0, pi]
        phi = 2*np.pi*np.random.rand()             # azimuthal angle [0, 2pi]

        # Construct state |ψ> = cos(theta/2)|0> + exp(i*phi)*sin(theta/2)|1>
        psi = np.cos(theta/2)*basis(2,0) + np.exp(1j*phi)*np.sin(theta/2)*basis(2,1)
        pure_dm = ket2dm(psi)

        # Random mixing with |1> to get mixed state
        p = np.random.rand()
        rho_mixed = p * pure_dm + (1-p) * ket2dm(basis(2,1))
        mixed_states.append(rho_mixed)
    return mixed_states

In [ ]:
mixed_state_data = mixed_state_generation()
print(mixed_state_data[0])

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.19136476+0.j         0.16308319+0.04733959j]
 [0.16308319-0.04733959j 0.80863524+0.j        ]]


In [ ]:
from qutip import Qobj, ket2dm, tensor, ptrace
import numpy as np # Import numpy

def generate_clones(mixed_state_data):
  cloned_pairs = []
  U_clone = Qobj([[1,0,0,0],
                [0,1/np.sqrt(2),1/np.sqrt(2),0],
                [0,1/np.sqrt(2),-1/np.sqrt(2),0],
                [0,0,0,1]], dims=[[2, 2], [2, 2]]) # Corrected dimensions
  ancilla = ket2dm(basis(2,0))          # Blank qubit |0>
  for rho in mixed_state_data:  # mixed_states is your list of 10k arrays
    rho_joint = tensor(rho, ancilla)      # Combine system

    rho_cloned = U_clone * rho_joint * U_clone.dag()  # apply unitary

    clone1 = ptrace(rho_cloned, 0)
    clone2 = ptrace(rho_cloned, 1)
    cloned_pairs.append((clone1, clone2))
  return cloned_pairs

In [ ]:
generated_clones = generate_clones(mixed_state_data)
print(generated_clones[0])

(Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.59568238+0.j         -0.11531723-0.03347414j]
 [-0.11531723+0.03347414j  0.40431762+0.j        ]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.59568238+0.j         0.11531723+0.03347414j]
 [0.11531723-0.03347414j 0.40431762+0.j        ]])


In [ ]:
import numpy as np
from qutip import sigmax, sigmay, sigmaz, Qobj

def qobj_to_bloch(rho):
    """Convert a single QuTiP Qobj density matrix to a Bloch vector."""
    x = np.real((rho * sigmax()).tr())
    y = np.real((rho * sigmay()).tr())
    z = np.real((rho * sigmaz()).tr())
    return np.array([x, y, z])

def convert_pairs_to_bloch(pair_list):
    """
    Convert a list of (rho1, rho2) pairs to Bloch vectors.

    Returns two arrays:
      - bloch1: first clones
      - bloch2: second clones
    """
    bloch1 = []
    bloch2 = []

    for rho1, rho2 in pair_list:
        bloch1.append(qobj_to_bloch(rho1))
        bloch2.append(qobj_to_bloch(rho2))

    return np.array(bloch1), np.array(bloch2)

# Example usage
# pair_list = [(rho1_clone1, rho1_clone2), (rho2_clone1, rho2_clone2), ...]
bloch_clone1, bloch_clone2 = convert_pairs_to_bloch(generated_clones)

print(bloch_clone1.shape)  # (10000, 3)
print(bloch_clone2.shape)  # (10000, 3)


(10000, 3)
(10000, 3)


In [ ]:
def convert_original_to_bloch(original_list):
    bloch = []

    for rho in original_list:
        bloch.append(qobj_to_bloch(rho))


    return np.array(bloch)

bloch_original = convert_original_to_bloch(mixed_state_data)

print(bloch_original.shape)  # (10000, 3)

(10000, 3)


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


x_train_orig, x_test_orig, x_train_c1, x_test_c1, x_train_c2, x_test_c2 = train_test_split(
    bloch_original, bloch_clone1, bloch_clone2, test_size=0.2, random_state=42
)

# Stack clones to create inputs and targets
X_train = np.vstack([x_train_c1, x_train_c2]).astype(np.float32)
y_train = np.vstack([x_train_orig, x_train_orig]).astype(np.float32)

X_test = np.vstack([x_test_c1, x_test_c2]).astype(np.float32)
y_test = np.vstack([x_test_orig, x_test_orig]).astype(np.float32)


# -------------------------------
# PyTorch Dataset
# -------------------------------
class BlochDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create datasets
train_dataset = BlochDataset(X_train, y_train)
test_dataset = BlochDataset(X_test, y_test)

# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch.nn as nn

class BlochDenoiser(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        return self.net(x)

model = BlochDenoiser()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 50

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    avg_loss = total_loss / len(train_loader.dataset)

    # Evaluate on test set
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            test_loss += criterion(output, y_batch).item() * X_batch.size(0)
    avg_test_loss = test_loss / len(test_loader.dataset)

    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {avg_loss:.6f} | Test Loss: {avg_test_loss:.6f}")

Epoch 1/50 | Train Loss: 0.132416 | Test Loss: 0.073341
Epoch 2/50 | Train Loss: 0.075432 | Test Loss: 0.071981
Epoch 3/50 | Train Loss: 0.075164 | Test Loss: 0.072008
Epoch 4/50 | Train Loss: 0.075152 | Test Loss: 0.071940
Epoch 5/50 | Train Loss: 0.075144 | Test Loss: 0.071745
Epoch 6/50 | Train Loss: 0.075142 | Test Loss: 0.071894
Epoch 7/50 | Train Loss: 0.075093 | Test Loss: 0.072209
Epoch 8/50 | Train Loss: 0.075098 | Test Loss: 0.071810
Epoch 9/50 | Train Loss: 0.075130 | Test Loss: 0.072008
Epoch 10/50 | Train Loss: 0.075098 | Test Loss: 0.071717
Epoch 11/50 | Train Loss: 0.075153 | Test Loss: 0.072397
Epoch 12/50 | Train Loss: 0.075114 | Test Loss: 0.071946
Epoch 13/50 | Train Loss: 0.075058 | Test Loss: 0.071977
Epoch 14/50 | Train Loss: 0.075114 | Test Loss: 0.072069
Epoch 15/50 | Train Loss: 0.075107 | Test Loss: 0.071899
Epoch 16/50 | Train Loss: 0.075074 | Test Loss: 0.071907
Epoch 17/50 | Train Loss: 0.075101 | Test Loss: 0.072486
Epoch 18/50 | Train Loss: 0.075073 | Tes

In [ ]:
import torch
import numpy as np

# Make sure the model is in evaluation mode
model.eval()

# Convert your test data to torch tensors if not already
X_test_tensor = torch.from_numpy(X_test)  # shape: (n_samples*2, 3)
y_test_tensor = torch.from_numpy(y_test)  # shape: (n_samples*2, 3)

# Run the model on test data
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Convert predictions back to NumPy
y_pred_np = y_pred.numpy()


In [ ]:
mse = np.mean((y_pred_np - y_test)**2)
print(f"Test MSE: {mse:.8f}")


Test MSE: 0.07205620


In [ ]:
import numpy as np

def bloch_fidelity_batch(rho_array, sigma_array):
    """
    Compute fidelities between two arrays of Bloch vectors.

    Parameters:
        rho_array: np.array of shape (n_samples, 3) - true vectors
        sigma_array: np.array of shape (n_samples, 3) - predicted vectors

    Returns:
        fidelities: np.array of shape (n_samples,)
    """
    r_dot = np.sum(rho_array * sigma_array, axis=1)           # dot product for each sample
    r_norm = np.linalg.norm(rho_array, axis=1)               # magnitude of true vectors
    s_norm = np.linalg.norm(sigma_array, axis=1)             # magnitude of predicted vectors
    fidelities = 0.5 * (1 + r_dot + np.sqrt(1 - r_norm**2) * np.sqrt(1 - s_norm**2))
    return fidelities

# Example usage:
fidelities_clone_orig = bloch_fidelity_batch(y_test, X_test)
fidelities_pred_true = bloch_fidelity_batch(y_test, y_pred_np)

# Average fidelity over all test samples
print(f"Average pre trained fidelity: {np.mean(fidelities_clone_orig):.6f}")

# Average fidelity over all test samples
print(f"Average post trained fidelity: {np.mean(fidelities_pred_true):.6f}")

# Optional: inspect first few
for i in range(5):
    print(f"True: {y_test[i]}, Predicted: {y_pred_np[i]}, Fidelity: {fidelities_pred_true[i]:.6f}")


Average pre trained fidelity: 0.710877
Average post trained fidelity: 0.930785
True: [ 0.14589943 -0.02331863 -0.8389816 ], Predicted: [ 0.00715271 -0.00645513 -0.84199   ], Fidelity: 0.995052
True: [ 0.17363155  0.1788907  -0.14037886], Predicted: [ 0.01014604 -0.00109477 -0.1430178 ], Fidelity: 0.984970
True: [-0.30914336  0.01048952 -0.5911604 ], Predicted: [ 0.00728942 -0.00947306 -0.5927216 ], Fidelity: 0.973954
True: [ 0.04740435  0.18015814 -0.29065287], Predicted: [ 0.00626321 -0.0084717  -0.2912472 ], Fidelity: 0.990600
True: [-0.08149549 -0.09591438 -0.9487776 ], Predicted: [ 0.00362801 -0.00326616 -0.95251316], Fidelity: 0.995985
